In [15]:
import os

# Fill the environmental variable with your own OpenAI API key
# See: https://platform.openai.com/account/api-keys
os.environ["OPENAI_API_KEY"] = "x"

In [16]:
import qdrant_client

client = qdrant_client.QdrantClient("http://localhost:6333", prefer_grpc=True)
client.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='KB-json-f')])

In [17]:
import pandas as pd

df = pd.read_csv("../output.csv")

FileNotFoundError: [Errno 2] No such file or directory: '../output.csv'

In [ ]:
df = df.head(1000)

In [ ]:
COLLECTION_NAME="KB-json-g"

In [ ]:
docs = df.to_json(orient='records', lines=True)

In [ ]:
docs = docs.split("\n")

In [ ]:
client.add(collection_name=COLLECTION_NAME, documents=docs)

['919217402a0849a896be8b82443e51ea',
 'f06671226d2b406bb23e5f5d9d3b6a48',
 '547d69cce2484107af6ee2f020767b01',
 '554759d7eaf54c75a7aecb9ccd10d641',
 'd18abab66fdb4b66865350ee7501a877',
 '4af3ab543ed840079bbcc922401b6ba4',
 '704029da173047af977da04a468deb7a',
 'b529a2e9cf9e4ad38698e805195fe510',
 '8597c82df52f4daf9d00d24aa5c9f963',
 '2cf85dcda97d4bf697ea57455e50c805',
 '09dc475e8d8e4d1190bf01c6692097c0',
 'd3e7312868b24ff1a899ac09aec0d463',
 '7cb9b2e774e244b2b793090d71894641',
 '35bf9f106a88437f91a27491b8548863',
 '89b6bf50438144a38e8aeb52642ccb88',
 '5ef624f91e5645fbb63947b322c7cec3',
 'f599a4e8128b49d98ae0103cf5ecf9ef',
 'df44d085f4594d6caab246f6de8a55d1',
 '902d9382e8454e73bf001ec4e129b818',
 '57029468d76b48f986fd9e8517c671a0',
 '5e6531ba7f4440328fba7b15a5f9aeb9',
 '2b519ccefe4142179db9a8f1c0d45c18',
 'e51376f9d331414fa1876de775b12152',
 '7e84241f65194458bfa8238e30d78530',
 '76d5dcde38dc4c40b137932d5b02e4fa',
 '08989511bbfe4b4ba14722ef1da31a1a',
 '19ab7c5d594944a38f5743cb925352ab',
 

In [ ]:
from openai import OpenAI

def rag(question: str, n_points: int = 30) -> str:
    results = client.query(
        collection_name=COLLECTION_NAME,
        query_text=question,
        limit=n_points,
    )

    context = "\n".join(r.document for r in results)

    metaprompt = f"""
    You are a software architect. 
    Answer the following question using the provided context. 
    If you can't find the answer, do not pretend you know it, but answer "I don't know".
    
    Question: {question.strip()}
    
    Context: you have the following list of Git Commits: {context.strip()}.
    
    Answer:
    """
    oaclient = OpenAI()
    completion = oaclient.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "user", "content": metaprompt},
        ],
        timeout=10.0,
    )
    return completion.choices[0].message.content

In [ ]:
rag("Tell me which files are modified the most", n_points=100)

"The file '.env.production.sample' is modified the most with a total of 6 modifications."